### Задание:

Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

Целевая переменная:
Price

Основная метрика:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Вспомогательная метрика:
MSE - средняя квадратичная ошибка (sklearn.metrics.mean_squared_error)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score as r2


%matplotlib inline
%config InlineBackend.figure_format = "svg"

In [2]:
train_dt = pd.read_csv("C:/Final_pjt/train.csv")
test_dt = pd.read_csv("C:/Final_pjt/test.csv")
train_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [3]:
test_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


Для начала удалим из тренировочный и тестовых даных признак "Healthcare_1" т.к. он мало заполнен и востановить его реальные значения практически не возможно:

In [4]:
train_dt = train_dt.drop("Healthcare_1", axis=1)
test_dt = test_dt.drop("Healthcare_1", axis=1)
test_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000


Далее постараемся разобратьс с этажностью зданий - признак "HouseFloor", который не может быть равен нулю. Напишим функцию, которая для всех наблюдений с "HouseFloor" будет меять этажность на среднюю по признаку "DistrictId":

In [5]:
def floor_processing(df:pd.DataFrame) -> pd.DataFrame:
    district_ind = df.loc[df["HouseFloor"] == 0, "DistrictId"].values[0]
    DI_ind, HF_ind, = df.columns.get_loc('DistrictId'), df.columns.get_loc('HouseFloor')
    for index in df.loc[df["HouseFloor"] == 0].index:
        DI = df.iloc[index, DI_ind]
        HF = round(df.loc[df['DistrictId'] == DI, "HouseFloor"].median())
        df.iloc[index, HF_ind] = HF if HF != 0 else df.iloc[index, "Floor"]
    return df

In [6]:
train_dt = floor_processing(train_dt)
test_dt = floor_processing(test_dt)
train_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.96610,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.45857,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,1.00000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.00000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,14.00000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.00000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.00000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


Для всех наблюдений, где признак "Floor" больше, чем признак "HouseFloor" приравняем этаж к максимальному этажу здания:

In [7]:
train_dt.loc[train_dt["Floor"] > train_dt["HouseFloor"], "HouseFloor"] = train_dt["Floor"]
test_dt.loc[test_dt["Floor"] > test_dt["HouseFloor"], "HouseFloor"] = test_dt["Floor"]
train_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,13.575000,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.008426,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,1.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,14.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


Далее модифицируем данные по квартирам с нулем комнат

In [8]:
train_dt.loc[train_dt["Rooms"] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1397,12638,27,0.0,138.427694,136.215499,0.0,4,4.0,2016,0.075424,B,B,11,3097,0,0,0,B,268394.744389
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,B,B,9,1892,0,0,1,B,302211.260887
2269,7317,27,0.0,41.790881,NaN,0.0,13,13.0,1977,0.211401,B,B,9,1892,0,0,1,B,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,16.0,2015,0.118537,B,B,30,6207,1,1,0,B,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,17.0,1977,0.243205,B,B,5,1564,0,0,0,B,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,12.0,1977,0.017647,B,B,2,469,0,0,0,B,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,B,B,43,8429,3,3,9,B,158998.110646
8834,9443,27,0.0,87.762616,85.125471,0.0,5,15.0,1977,0.211401,B,B,9,1892,0,0,1,B,219281.918007


In [9]:
test_dt.loc[test_dt["Rooms"] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2406,3343,58,0.0,116.824201,113.692424,0.0,3,3.0,1977,0.437885,B,B,23,5735,3,0,5,B
2524,10729,27,0.0,76.345154,42.820796,12.0,14,14.0,1977,0.017647,B,B,2,469,0,0,0,B


Напишим отдельную функцию для определения "реального" числа комнат для квартир со значением "Rooms" = 0, используя функцию "zero_rooms_squere"

In [10]:
def zero_rooms_change(df:pd.DataFrame, koef:float=0.1) -> pd.DataFrame:
    """Функция принимае датафрейм и для всех объектов у которых число комнат равно 0 по площаде квартиры вычисляет число комнат:
    определяет площадь текущей квартиры (squere), делает выборку квартир с площадью в диапазоне от 0,9*squere до 1,1*squere,
    итоговая площадь равна среднему из полученной выбоки"""
    square_ind, room_ind, = df.columns.get_loc('Square'), df.columns.get_loc('Rooms')
    for index in df.loc[df["Rooms"] == 0].index:
        square = df.iloc[index, square_ind]
        room = df.loc[((df["Square"] > square * (1 - koef)) & (df["Square"] < square * (1 + koef))), "Rooms"].median()
        df.iloc[index, room_ind] = round(room) if round(room) != 0 else 1.0
    return df

In [11]:
train_dt = zero_rooms_change(train_dt, 0.1)
test_dt = zero_rooms_change(test_dt, 0.1)
train_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.892300,56.315775,37.199645,6.273300,8.526700,13.575000,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.838434,21.058732,86.241209,28.560917,5.241148,6.008426,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,1.136859,0.370619,0.000000,1.000000,1.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,14.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


Предположим, что квартир с площадью менее 15 м2 быть не может:

In [12]:
train_dt.loc[train_dt["Square"] < 15]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
212,1748,88,2.0,5.497061,67.628717,1.0,24,24.0,1977,0.127376,B,B,43,8429,3,3,9,B,412511.088764
1316,11526,27,1.0,4.633498,1.969969,1.0,18,18.0,1977,0.011654,B,B,4,915,0,0,0,B,107604.269441
1608,10202,6,1.0,2.596351,4.604943,1.0,3,25.0,2014,0.243205,B,B,5,1564,0,0,0,B,137597.601458
3280,10527,27,1.0,4.380726,40.805837,1.0,10,17.0,2013,0.211401,B,B,9,1892,0,0,1,B,97560.720383
3413,9487,5,1.0,5.129222,5.549458,1.0,1,1.0,1977,0.150818,B,B,16,3433,4,4,5,B,369472.403061
4739,12676,81,3.0,13.784865,15.988889,7.0,4,5.0,1960,0.319809,B,B,25,4756,16,5,8,B,78388.806186
4853,3224,27,1.0,2.377248,0.873147,0.0,1,12.0,1977,0.017647,B,B,2,469,0,0,0,B,126596.941798
4900,4504,27,3.0,4.390331,5.610772,1.0,8,19.0,2016,0.211401,B,B,9,1892,0,0,1,B,161379.067034
6392,14786,1,1.0,1.136859,4.525736,1.0,3,3.0,1977,0.007122,B,B,1,264,0,0,1,B,181434.825589
8030,13265,1,3.0,4.823679,79.767964,0.0,6,17.0,1977,0.007122,B,B,1,264,0,0,1,B,237716.681261


In [13]:
test_dt.loc[test_dt["Square"] < 15]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,5,5,B
66,9011,53,1.0,1.378543,1.353573,1.0,1,1.0,1977,0.049637,B,B,34,7759,0,1,3,B
608,16401,30,1.0,2.645046,4.338755,1.0,2,2.0,1977,0.000078,B,B,22,6398,141,3,23,B
837,2138,27,1.0,5.647458,1.501582,1.0,1,1.0,1977,0.017647,B,B,2,469,0,0,0,B
1165,10120,6,1.0,5.100672,3.861780,1.0,3,3.0,1977,0.243205,B,B,5,1564,0,0,0,B
3697,15435,21,1.0,13.305531,9.495367,0.0,6,6.0,1965,0.194489,B,B,47,8004,3,3,5,B
4490,1165,27,1.0,2.372101,1.899119,1.0,2,17.0,1977,0.011654,B,B,4,915,0,0,0,B
4540,7855,6,1.0,4.967143,2.968086,1.0,3,3.0,2018,0.243205,B,B,5,1564,0,0,0,B
4713,170,6,2.0,2.900586,61.468563,1.0,18,18.0,2014,0.243205,B,B,5,1564,0,0,0,B
4902,1420,45,1.0,1.975769,2.900371,1.0,1,1.0,1977,0.195781,B,B,23,5212,6,3,2,B


Напишим отдельную функцию для определения "реальной" плоцади для квартир со значением "Squere" < 15, используя функцию "zero_square_change"

In [14]:
def zero_square_change(df:pd.DataFrame, min_square:float, koef:float=0.05) -> pd.DataFrame:
    """Функция принимае датафрейм и для всех объектов у которых площадь меньше значения "min_squere" по числу комнат вычисляет
    площадь: определяет число комнат текущей квартиры (rooms), делает выборку квартир с числом комнат в диапазоне от (1-koef)*rooms
    до (1+koef)*rooms, итоговая площадь равна среднему из полученной выбоки"""
    square_ind, room_ind, = df.columns.get_loc('Square'), df.columns.get_loc('Rooms')
    for index in df.loc[df["Square"] < min_square].index:
        rooms = df.iloc[index, room_ind]
        square = df.loc[((df["Rooms"] > rooms * (1 - koef)) & (df["Rooms"] < rooms * (1 + koef))), "Square"].median()
        df.iloc[index, square_ind] = square if square > min_square else min_squere
    return df

In [15]:
train_dt = zero_square_change(train_dt, 15)
test_dt = zero_square_change(test_dt, 15)
train_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.892300,56.375134,37.199645,6.273300,8.526700,13.575000,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.838434,20.983872,86.241209,28.560917,5.241148,6.008426,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,16.117154,0.370619,0.000000,1.000000,1.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.783182,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.537977,32.781260,6.000000,7.000000,14.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.925548,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570


Создадим функцию, чтобы посмотреть и удалить выбросы для признаков "Rooms" и "Square"

In [16]:
def clean_dt(trn_df:pd.DataFrame, tst_df:pd.DataFrame, feat_list:list, lim_coef:list=[10.0, 10.0]) -> tuple:
    """Функция для предобработки исходных данных - получает на входе список из двух признаков 'feat_list', строит график
    зависимости второго признака от первого, после чего обрабатывает выбросы - все значения признака, которые больше, чем
    среднее значение этого признака помноженного на соответствующий коэффициент из 'lim_coef' приравниваются к среднему.
    'lim_coef[0]' содержит значения коэф. для 1-го признака, 'lim_coef[1]' - для второго. После обработки снова выводит графики
    для оценки результата. Функция возвращает кортеж обработанных датафреймов"""
    
    trn_ft0_med, trn_ft0_min, trn_ft0_max = trn_df[feat_list[0]].median(), trn_df[feat_list[0]].min(), trn_df[feat_list[0]].max()
    trn_ft1_med, trn_ft1_min, trn_ft1_max = trn_df[feat_list[1]].median(), trn_df[feat_list[1]].min(), trn_df[feat_list[1]].max()
    tst_ft0_med, tst_ft0_min, tst_ft0_max = tst_df[feat_list[0]].median(), tst_df[feat_list[0]].min(), tst_df[feat_list[0]].max()
    tst_ft1_med, tst_ft1_min, tst_ft1_max = tst_df[feat_list[1]].median(), tst_df[feat_list[1]].min(), tst_df[feat_list[1]].max()

    fig, ax = plt.subplots(nrows=2, ncols=2)
    fig.set_size_inches(12, 8)
    plt.subplots_adjust(wspace=0.3, hspace=0.3)
    ax_train, ax_test, ax_train_aft, ax_test_aft = ax.flatten()
    ax_train.set_title("Тренировочные данные ДО")
    ax_train.set_xlabel(feat_list[0])
    ax_train.set_ylabel(feat_list[1])
    ax_train.scatter(trn_df[feat_list[0]], trn_df[feat_list[1]])
    ax_train.vlines(trn_ft0_med, trn_ft1_min, trn_ft1_max, label=str(trn_ft0_med))
    ax_train.hlines(trn_ft1_med, trn_ft0_min, trn_ft0_max, label=str((trn_ft1_med)))
    ax_train.legend()            
    ax_test.set_title("Тестовые данные ДО")
    ax_test.set_xlabel(feat_list[1])
    ax_test.scatter(tst_df[feat_list[0]], tst_df[feat_list[1]])
    ax_test.vlines(tst_ft0_med, tst_ft1_min, tst_ft1_max, label=str(tst_ft0_med))
    ax_test.hlines(tst_ft1_med, tst_ft0_min, tst_ft0_max, label=str((tst_ft1_med)))
    ax_test.legend()
    
    trn_df.loc[trn_df[feat_list[0]] > (trn_ft0_med * lim_coef[0]), feat_list[0]] = trn_ft0_med
    trn_df.loc[trn_df[feat_list[1]] > (trn_ft1_med * lim_coef[1]), feat_list[1]] = trn_ft1_med
    tst_df.loc[tst_df[feat_list[0]] > (tst_ft0_med * lim_coef[0]), feat_list[0]] = tst_ft0_med
    tst_df.loc[tst_df[feat_list[1]] > (tst_ft1_med * lim_coef[1]), feat_list[1]] = tst_ft1_med
    
    ax_train_aft.set_title("Тренировочные данные ПОСЛЕ")
    ax_train_aft.set_xlabel(feat_list[0])
    ax_train_aft.set_ylabel(feat_list[1])
    ax_train_aft.scatter(trn_df[feat_list[0]], trn_df[feat_list[1]])
    ax_test_aft.set_title("Тестовые данные ПОСЛЕ")
    ax_test_aft.set_xlabel(feat_list[1])
    ax_test_aft.scatter(tst_df[feat_list[0]], tst_df[feat_list[1]])
    
    return (trn_df, tst_df)

Подбираем коэффициенты так, чтобы удалить выбросы, итоговый значения коэффициентов - это 4 для "Rooms" и 7 для "Squere"

In [17]:
train_dt, test_dt = clean_dt(train_dt, test_dt, ["Rooms", "Square"], lim_coef=[4, 7.0])

Заменим даты постройки дома ""HouseYear" на реальные значения:

In [18]:
train_dt.loc[train_dt["HouseYear"] == 20052011, "HouseYear"] = 2008
train_dt.loc[train_dt["HouseYear"] == 4968, "HouseYear"] = 1968
train_dt.loc[train_dt["HouseYear"] > 2020]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price


Вычислим цену квадратного метра для каждой квартиры

In [19]:
train_dt["price_by_meter"] = train_dt["Price"]/train_dt["Square"]
train_dt.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,price_by_meter
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,0,11,B,184966.930730,3854.958571
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,1,16,B,300009.450063,4567.491245
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,1,3,B,220925.908524,4915.149522
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,0,5,B,175616.227217,3291.591637
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2,4,B,150226.531644,3788.892642


Определим среднюю цену квадратного метра для квартир в каждом районе, дополнительно крупирую по количеству комнат. Также сразу определим среднюю стоимость метра по всей выборке, для заполнения значений NaN

In [20]:
price_by_meter_DI = train_dt.groupby(["DistrictId", "Rooms"], as_index=False)["price_by_meter"].mean()
price_by_meter_DI.rename(columns={"price_by_meter":"pbm"}, inplace=True)
mean_price = price_by_meter_DI["pbm"].mean()
price_by_meter_DI.head()

,DistrictId,Rooms,pbm
0,0,1.0,3743.512621
1,0,2.0,3631.502943
2,0,3.0,3774.993359
3,1,1.0,3388.841270
4,1,2.0,3159.351031


In [21]:
train_dt.loc[(train_dt["DistrictId"] == 58) & (train_dt["Rooms"] == 2), "price_by_meter"].mean()

2962.6044533019599

Добавим данные о цене квадратного метра в общую тренировочную выборку

In [22]:
train_dt = pd.merge(train_dt, price_by_meter_DI, on=["DistrictId", "Rooms"], how="left")
train_dt.drop("price_by_meter", axis=1, inplace=True)
train_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Price,pbm
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.889000,56.225376,37.199645,6.273300,8.526700,13.575000,1984.86600,0.118858,24.687000,5352.157400,8.039200,1.319500,4.231300,214138.857399,3892.769353
std,4859.01902,43.587592,0.812739,19.067770,86.241209,28.560917,5.241148,6.008426,18.41187,0.119025,17.532614,4006.799803,23.831875,1.493601,4.806341,92872.293865,993.210374
min,0.00000,0.000000,1.000000,16.117154,0.370619,0.000000,1.000000,1.000000,1910.00000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,59174.778028,810.825491
25%,4169.50000,20.000000,1.000000,41.783182,22.769832,1.000000,4.000000,9.000000,1974.00000,0.017647,6.000000,1564.000000,0.000000,0.000000,1.000000,153872.633942,2980.325852
50%,8394.50000,36.000000,2.000000,52.537283,32.781260,6.000000,7.000000,14.000000,1977.00000,0.075424,25.000000,5285.000000,2.000000,1.000000,3.000000,192269.644879,3994.518464
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2001.00000,0.195781,36.000000,7227.000000,5.000000,2.000000,6.000000,249135.462171,4541.566010
max,16798.00000,209.000000,6.000000,275.645284,7480.592129,2014.000000,42.000000,117.000000,2020.00000,0.521867,74.000000,19083.000000,141.000000,6.000000,23.000000,633233.466570,11854.652892


Добавим данные о цене квадратного метра в тесовую выборку

In [24]:
test_dt = pd.merge(test_dt, price_by_meter_DI, on=["DistrictId", "Rooms"], how="left")
test_dt.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,pbm
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,0,0,B,2962.604453
1,15856,74,2.0,69.263183,NaN,1.0,6,6.0,1977,0.075779,B,B,6,1437,3,0,2,B,3289.878362
2,5480,190,1.0,40.117689,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,5,5,B,NaN
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,3,3,B,3841.195532
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,0,0,A,2886.882245


Сразу видно, что появились пропущенные данные. Заменим их средней стоимостью квадратного метра

In [26]:
test_dt["pbm"] = test_dt["pbm"].fillna(value=mean_price)
test_dt.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,pbm
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,8412.595400,51.279200,1.908200,56.522100,36.158810,5.976800,8.632000,13.537400,1984.392600,0.119874,24.933800,5406.900000,8.262600,1.31940,4.242800,3911.901478
std,4832.674037,44.179466,0.810373,18.967286,17.825287,9.950018,5.483228,6.157612,18.573149,0.120070,17.532202,4026.614773,23.863762,1.47994,4.777365,978.196725
min,1.000000,0.000000,1.000000,16.319015,0.333490,0.000000,1.000000,1.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.00000,0.000000,1016.753916
25%,4221.750000,21.000000,1.000000,41.912403,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,0.00000,1.000000,3025.145750
50%,8320.500000,37.000000,2.000000,52.924953,32.925087,6.000000,7.000000,14.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,1.00000,3.000000,4004.094401
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,2.00000,6.000000,4583.611703
max,16795.000000,212.000000,6.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,6.00000,23.000000,8815.344841


Попробуем обучить модель используя RandomForest. Удалим лишние данные

In [27]:
target = train_dt["Price"]
feautures = train_dt.drop(["LifeSquare", "KitchenSquare", "Ecology_2", "Ecology_3", "Shops_2",\
                           "Social_1", "Price"], axis=1)
feautures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 13 columns):
Id             10000 non-null int64
DistrictId     10000 non-null int64
Rooms          10000 non-null float64
Square         10000 non-null float64
Floor          10000 non-null int64
HouseFloor     10000 non-null float64
HouseYear      10000 non-null int64
Ecology_1      10000 non-null float64
Social_2       10000 non-null int64
Social_3       10000 non-null int64
Helthcare_2    10000 non-null int64
Shops_1        10000 non-null int64
pbm            10000 non-null float64
dtypes: float64(5), int64(8)
memory usage: 1.1 MB


Разобъем выборку на тренировочный и проверочный наборы

In [29]:
X_train, X_test, y_train, y_test = train_test_split(feautures, target, test_size=30, random_state=40)

Попытаемся найти наилучшие параметры для модели RandomForestRegressor

In [33]:
parameters = [{"n_estimators" : [200, 300, 400]},
            {"max_depth" : np.arange(10, 15)},
             {"min_samples_leaf": np.arange(2, 5)}]

In [36]:
clf = GridSearchCV(estimator=RF(random_state=40),
                  param_grid=parameters,
                  scoring="r2",
                  cv=3)

In [37]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=40, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [200, 300, 400]}, {'max_depth': array([10, 11, 12, 13, 14])}, {'min_samples_leaf': array([2, 3, 4])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [38]:
clf.best_params_

{'n_estimators': 200}

Но почему-то не получилось)))

In [39]:
model = RF(n_estimators=400, max_depth=15, min_samples_leaf=3)
# model = RF(n_estimators=700, max_depth=27, min_samples_leaf=4),

In [40]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [43]:
y_pred = model.predict(X_test)
r2(y_test, y_pred)

0.83469961467721765

In [45]:
y_pred_trn = model.predict(X_train)
r2(y_train, y_pred_trn)

0.91710987626398033

Возможно переобучение, но вроде сойдет)
Обработаем тестовые данные

In [46]:
test_feautures = test_dt.drop(["LifeSquare", "KitchenSquare", "Ecology_2", "Ecology_3", "Shops_2",\
                               "Social_1",], axis=1)
test_feautures.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 13 columns):
Id             5000 non-null int64
DistrictId     5000 non-null int64
Rooms          5000 non-null float64
Square         5000 non-null float64
Floor          5000 non-null int64
HouseFloor     5000 non-null float64
HouseYear      5000 non-null int64
Ecology_1      5000 non-null float64
Social_2       5000 non-null int64
Social_3       5000 non-null int64
Helthcare_2    5000 non-null int64
Shops_1        5000 non-null int64
pbm            5000 non-null float64
dtypes: float64(5), int64(8)
memory usage: 546.9 KB


In [50]:
test_dt["Price"] = model.predict(test_feautures)

In [52]:
test_dt.loc[:,["Id", "Price"]].to_csv("AGavrilov_predictions.csv", index=None)